Aşağıda ihtiyacımız doğrultusunda kullanacağımız kütüphaneleri yükleyelim.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Veri çerçevemizi bulunduğumuz dizinden yükleyelim ve bir veri çerçevesi haline getirerek df değişkenine atayalım. (pd.read_csv(...csv))

In [ ]:
df=pd.read_csv("../input/tabletpc-priceclassification/tablet.csv")

Veri çerçevesinin ilk 5 gözlemini görüntüleyelim.

In [ ]:
df.head()

Veri çerçevesinin kaç öznitelik ve kaç gözlemden oluştuğunu görüntüleyelim.

In [ ]:
df.shape

Veri çerçevesindeki değişkenlerin hangi tipte olduğunu ve bellek kullanımını görüntüleyelim.

In [ ]:
df.info()

Veri çerçevesindeki sayısal değişkenler için temel istatistik değerlerini görüntüleyelim.

Standart sapma ve ortalama değerlerden çıkarımda bulunarak hangi değişkenlerin ne kadar varyansa sahip olduğu hakkında fikir yürütelim.

In [ ]:
df.dtypes

Daha sonra dolduracağımız boş değerlere bir göz atalım.

In [ ]:
df.isna().sum()

Verinin genel sayısal istatistiklerine bakalım

In [ ]:
df.describe().T

In [ ]:
df.cov()

Sayısal değişkenler arasında korelasyon olup olmadığını göstermek için korelasyon matrisi çizdirelim. Korelasyon katsayıları hakkında fikir yürütelim.


In [ ]:
corr=df.corr()
corr

Görüldüğü gibi pozitif olarak en güçlü iki ilişki ArkaKameraMP ve OnKameraMP arasındadır

Korelasyon katsayılarını daha iyi okuyabilmek için ısı haritası çizdirelim.

In [ ]:
sns.heatmap(df.corr())

Dağılım grafiğini çizdiriyoruz.

In [ ]:
sns.scatterplot(x = df["ArkaKameraMP"], y = df["OnKameraMP"])

Diğer değişkenimizinde dağılım grafiğini çiziyoruz fakat üst üste binen verileri daha rahat okuyabilmek için yoğunluklu dağılım grafiği çizdiriyoruz.

In [ ]:
sns.jointplot(x = df["CozunurlukYükseklik"], y = df["CozunurlukGenislik"])

Hedef değişkenimizin benzersiz değerini kontrol ediyoruz.

In [ ]:
df['FiyatAraligi'].unique()

In [ ]:
df['FiyatAraligi'].nunique()

In [ ]:
df['FiyatAraligi'].value_counts()

Değişkenimiz eşit dağılmış.

Veriyi daha iyi anlamak için çeşitli değişkenleri grafiklere dökerek inceliyoruz.

In [ ]:
sns.distplot(df["BataryaGucu"], bins=40, color="purple");

In [ ]:
sns.countplot(x="Bluetooth",data=df);

In [ ]:
sns.barplot(x="FiyatAraligi",y="ArkaKameraMP",data=df);

In [ ]:
sns.barplot(x="FiyatAraligi",y="RAM",data=df);

Buradan RAM'in fiyatı doğrudan etkilediği çıkarımını yapabiliriz.

In [ ]:
sns.barplot(x="FiyatAraligi",y="MikroislemciHizi",data=df);

İşlemci hızının fiyata çok etki etmediğini görüyoruz

In [ ]:
sns.violinplot(y="MikroislemciHizi",data=df);

Genellikle düşük GHZ'ye sahip mikroişlemciler kullanılmış.

In [ ]:
sns.violinplot(y="BataryaGucu",x="WiFi",data=df);

In [ ]:
sns.jointplot(x=df["BataryaGucu"],y=df["BataryaOmru"],kind="kde",color="purple")

Boşluk ya da "Girilmedi" gibi boş değerler var mı kontrol ediyoruz.

In [ ]:
df["Bluetooth"].unique()

In [ ]:
df["CiftHat"].unique()

In [ ]:
df["MikroislemciHizi"].unique()

In [ ]:
df["OnKameraMP"].unique()

In [ ]:
df["4G"].unique()

In [ ]:
df["Kalinlik"].unique()

In [ ]:
df["CekirdekSayisi"].unique()

In [ ]:
df["ArkaKameraMP"].unique()

In [ ]:
df["3G"].unique()

In [ ]:
df["Dokunmatik"].unique()

In [ ]:
df["WiFi"].unique()

In [ ]:
df["FiyatAraligi"].unique()

In [ ]:
df["Renk"].unique()

In [ ]:
msno.heatmap(df,figsize=(10,5))

Bu grafikten eksik verilerin aynı verilerde olmadığını anlayabiliriz.

In [ ]:
df['WiFi'] = df['WiFi'].map({'Var': 1, 'Yok': 0})
df['Bluetooth'] = df['Bluetooth'].map({'Var': 1, 'Yok': 0})
df['CiftHat'] = df['CiftHat'].map({'Var': 1, 'Yok': 0})
df['4G'] = df['4G'].map({'Var': 1, 'Yok': 0})
df['3G'] = df['3G'].map({'Var': 1, 'Yok': 0})
df['Dokunmatik'] = df['Dokunmatik'].map({'Var': 1, 'Yok': 0})

Yok yerine 0 , Var yerine 1 yazdık. NOT: Tam tersini yaptığı için LabelEncoder kullanmadık.

In [ ]:
df.head()

**Bazı Firmalar renklere göre fiyat politikası uyguladığı için renkleri verisetimizden silmiyoruz**

In [ ]:
df["Renk"] = pd.Categorical(df["Renk"])
RenkDummies = pd.get_dummies(df["Renk"], prefix = "Renk")
df = pd.concat([df,RenkDummies],axis = 1)
df.head()

In [ ]:
df.drop(["Renk","Renk_Kahverengi"],axis = 1, inplace = True)
df.head()

In [ ]:
categories=pd.Categorical(df['FiyatAraligi'],categories=['Çok Ucuz','Ucuz','Normal','Pahalı'],ordered=True)
FiyatAraligi,unique=pd.factorize(categories,sort=True)
df['FiyatAraligi']=FiyatAraligi


In [ ]:
df['FiyatAraligi']

In [ ]:
df[df["RAM"].isnull()][["RAM","FiyatAraligi","CekirdekSayisi","MikroislemciHizi","DahiliBellek", "Bluetooth"]]

In [ ]:
df[df["FiyatAraligi"] == 3]["RAM"].mean()
df[df["RAM"].isnull()][["RAM","FiyatAraligi","CekirdekSayisi","MikroislemciHizi","DahiliBellek", "Bluetooth"]]

Fiyat aralığı Ram ile doğrusal bir ilişkide olduğu için ve boş elamanların hepsi Pahalı Fiyat aralığında sahip olduğu için boş değerleri Pahalı fiyat aralığı değerlerinin ortalaması ile dolduruyoruz.

In [ ]:
df["RAM"].fillna(3449.3504098360654, inplace = True) 

In [ ]:
df[df["RAM"].isnull()][["OnKameraMP","FiyatAraligi","CekirdekSayisi","MikroislemciHizi","DahiliBellek","Bluetooth"]]

Boş veri kalmadığını görüyoruz.

In [ ]:
df[df["FiyatAraligi"] == 0]["OnKameraMP"].mean() # Ucuz telefonların ortalaması

In [ ]:
df[df["FiyatAraligi"] == 1]["OnKameraMP"].mean()

In [ ]:
df[df["FiyatAraligi"] == 2]["OnKameraMP"].mean()

In [ ]:
df[df["FiyatAraligi"] == 3]["OnKameraMP"].mean()

In [ ]:
df["OnKameraMP"].mean() # Tüm telefonların ortalaması

In [ ]:
df[df["FiyatAraligi"] == 3][["ArkaKameraMP","OnKameraMP"]].mean()

In [ ]:
df[df["FiyatAraligi"] == 2][["ArkaKameraMP","OnKameraMP"]].mean()

In [ ]:
df[df["FiyatAraligi"] == 1][["ArkaKameraMP","OnKameraMP"]].mean()

In [ ]:
df[df["FiyatAraligi"] == 0][["ArkaKameraMP","OnKameraMP"]].mean() # Ucuz telefonların onkamera ve arka kamera ortalaması

Fiyat Aralığı ile OnKameraMP'nin herhangi bir ilişkisi olmadığını gördük. Bu yüzden Arka kameranın değerine göre ön kameranın boş değerlerini doldurduk. 

In [ ]:
df["OnKameraMP"].fillna(4.095123, inplace = True) 
df[df["OnKameraMP"].isnull()][["RAM","FiyatAraligi","ArkaKameraMP","MikroislemciHizi","DahiliBellek", "Bluetooth"]]

Boş veri kalmadığını görüyoruz.

In [ ]:
X = df.drop("FiyatAraligi",axis = 1)
Y = df["FiyatAraligi"]

In [ ]:
X

In [ ]:
Y

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 1/4)

In [ ]:
X_train.count()

In [ ]:
X_test.count()

In [ ]:
Y_train.count()

In [ ]:
Y_test.count()

In [ ]:
Y_train

In [ ]:
modeller = { 
    'Gercek Degerler': []
}


In [ ]:
dfModel = pd.DataFrame(modeller, columns = ['Gercek Degerler'])

Modelleri ilerde karşılaştırmak için bir dataframe oluşturduk.

In [ ]:
clf = GaussianNB()

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
y_pred_gaussian = clf.predict(X_test)


In [ ]:
clf_decisiontree = DecisionTreeClassifier()


In [ ]:
clf_train_decisiontree = clf_decisiontree.fit(X_train, Y_train)

In [ ]:
y_pred_decisiontree = clf_train_decisiontree.predict(X_test)

In [ ]:
KNNeigh = KNeighborsClassifier()

In [ ]:
KNNeigh.fit(X_train, Y_train)

In [ ]:
y_pred_KNNeigh = KNNeigh.predict(X_test)

**KNeighborsClassifier**

In [ ]:
print(classification_report(Y_test , y_pred_KNNeigh))


In [ ]:
confusion_matrix(Y_test , y_pred_KNNeigh)

**GaussianNB**

In [ ]:
print(classification_report(Y_test, y_pred_gaussian))


In [ ]:
confusion_matrix(Y_test, y_pred_gaussian)

**DecisionTreeClassifier**

In [ ]:
print(classification_report(Y_test, y_pred_decisiontree))



In [ ]:
confusion_matrix(Y_test, y_pred_decisiontree)

In [ ]:
decision = classification_report(Y_test , y_pred_decisiontree,output_dict="true") # ilerde criterion='entropy' olan değişkenle karşılaştırmak için dict şeklinde aldık

In [ ]:
dfModel["Gercek Degerler"] = df["FiyatAraligi"]
dfModel["GaussianModel"] = clf.predict(X)
dfModel["DecisionTreeModel"] = clf_decisiontree.predict(X)
dfModel["KNN_Model"] = neigh.predict(X)
dfModel.sample(10)

**GaussianModel Grafiği**

In [ ]:
sns.lmplot(x = "Gercek Degerler", y = "GaussianModel", data = dfModel); 

**DecisionTreeModel Grafiği**

In [ ]:
sns.lmplot(x = "Gercek Degerler", y = "DecisionTreeModel", data = dfModel); 

**KNN_Model Grafiği**

In [ ]:
sns.lmplot(x = "Gercek Degerler", y = "KNN_Model", data = dfModel); 

Çıkan değerlere göre en başarılı model KNeighborsClassifier algoritmasıyla eğittiğimizdir.

En başarısız model GaussianNB algoritmasıyla eğittiğimiz modeldir.

In [ ]:
clf_decisiontree = DecisionTreeClassifier(criterion='entropy')

In [ ]:
clf_train_decisiontree = clf_decisiontree.fit(X_train, Y_train)

In [ ]:
y_pred_decision_entropy = clf_train_decisiontree.predict(X_test)

In [ ]:
dfModel["DecisionTreeModel_Entropy"] = clf_decisiontree.predict(X)
dfModel.sample(10)[["Gercek Degerler","DecisionTreeModel","DecisionTreeModel_Entropy"]]

In [ ]:
print(classification_report(Y_test , y_pred_decision_gini))
decision_yeni = classification_report(Y_test , y_pred_decision_gini,output_dict="true")

In [ ]:
karsilastirma = { 
         'DecisionTreeClassifier': ['Gini','Entropy'],
         'Precision': [decision_yeni.get('weighted avg').get('precision') , decision_eski.get('weighted avg').get('precision')],
         'Recall': [decision_yeni.get('weighted avg').get('recall') , decision_eski.get('weighted avg').get('recall')],
         'F1-score': [decision_yeni.get('weighted avg').get('f1-score') , decision_eski.get('weighted avg').get('f1-score')],
         'Accuracy': [decision_yeni.get('accuracy') , decision_eski.get('accuracy')]
        }


In [ ]:
df_Decision_Gini = pd.DataFrame(karsilastirma, columns = ['DecisionTreeClassifier','Precision','Recall','F1-score', 'Accuracy'])
df_Decision_Gini

criterion='entropy' nin yarattığı etkileri tabloda net bir şekilde görebiliriz. Criterion değişkeninin Entropy olması modelin doğruluk başarısını ciddi bir şekilde düşürmüştür.

In [ ]:
knn_karsilastirma = {
         'Neighbors': [],
         'Precision': [],
         'Recall': [],
         'F1-score': [],
         'Accuracy': []
        }


In [ ]:
df_knn_karsilastirma = pd.DataFrame(knn_karsilastirma, columns = ['Neighbors','Precision','Recall','F1-score', 'Accuracy'])

In [ ]:
for x in range(2,16):
    neigh = KNeighborsClassifier(n_neighbors=x)
    neigh.fit(X_train, Y_train)
    y_pred =  neigh.predict(X_test)
    knn_report = classification_report(Y_test , y_pred,output_dict="true")
    df_knn_karsilastirma.loc[x-2] =[x,knn_report.get('weighted avg').get('precision'),knn_report.get('weighted avg').get('recall'),knn_report.get('weighted avg').get('f1-score'),knn_report.get('accuracy')]
    

In [ ]:
df_knn_karsilastirma

In [ ]:
sns.barplot(y="Accuracy",x="Neighbors",data=df_knn_karsilastirma)

In [ ]:
sns.barplot(y="F1-score",x="Neighbors",data=df_knn_karsilastirma)

In [ ]:
sns.barplot(y="Recall",x="Neighbors",data=df_knn_karsilastirma)

In [ ]:
sns.barplot(y="Precision",x="Neighbors",data=df_knn_karsilastirma)

Bu tablolara göre komşu sayısı artıkkça modelin başarısının da artığını söyleyebiliriz. Ancak her zaman için doğru değildir